In [121]:
import pandas as pd

df = pd.read_excel('retrieval_file_7.xlsx')

questions = df.questions
#predictions = df['generated_answers']
#references = df['answers']

In [122]:
df.loc[13]

Unnamed: 0                                                         999
question_ids                                                       999
bad                                                              False
questions            What is the most effective treatment against i...
contexts             Mucosal Vaccination with Recombinant Lactobaci...
answers                                                    Vaccination
bad_here                                                           NaN
retrieved_docs       ['in the treatment of influenza A   virus infe...
retrieved_doc_ids    ['cf2f1520-e52d-11ee-92f1-00155da004c0', 'cf2f...
Name: 13, dtype: object

In [116]:
df.loc[13].questions

'What are important criteria for selecting vaccine delivery vectors?'

In [118]:
df.loc[13].retrieved_docs

"['delivery vector, led to the widespread testing of this bacterium as a mucosal delivery system for antigens from non-Salmonella pathogens  . These studies have demonstrated the utility of live bacteria to deliver expressed antigens and DNA vaccines to the host immune system  . Since then several other intracellular bacterial vectors have been successfully tested for their capability to deliver a variety of antigens from various pathogens, as well as vaccination against cancer. One genus which has been widely tested as vector is Listeria. Listeria species are Gram-positive intracellular food-borne pathogens. The advantages of Listeria are that it can invade a variety of cells, including antigen presenting cells  . After invading the host cell, Listeria resides inside the phagosome; however,', 'as antigen delivery systems, including poxviruses, adenoviruses baculovirus, paramyxovirus, rhabdovirus, and others; however, the majority of human clinical trials assessing viral-vectored influ

In [14]:
predictions

0                                        viral infection
1      arenaviruses and dengue viruses, and more rece...
2                                       the upper airway
3                                               Listeria
4                                  pre-existing immunity
                             ...                        
282    The reporting rate was difficult to determine ...
283    Incubation period and latent period of human i...
284                                     infected rodents
285                                                 30 %
286                                            v-ATPases
Name: generated_answers, Length: 287, dtype: object

In [15]:
references

0      the synergistic effect of viral infection with...
1                        arenaviruses and dengue viruses
2                                       the upper airway
3      attenuated bacteria, including Escherichia col...
4      pre-existing immunity is a major obstacle of m...
                             ...                        
282    Because of the many concerns raised by the int...
283    It was found that the respiratory specimens we...
284    mucosal exposure to aerosols or by direct cont...
285                                                 30 %
286                      a family of AAA-ATPase proteins
Name: answers, Length: 287, dtype: object

In [1]:
import evaluate

rouge = evaluate.load('rouge', rouge_types=['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_aggregator=True, use_stemmer=True)
bleu = evaluate.load('bleu', max_order=4, smooth=False)
meteor = evaluate.load('meteor')

/home/samsatp/samsatp/RAG-QA/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to /home/samsatp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/samsatp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/samsatp/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
bleu.compute(predictions=['x'],references=['hello world this is good'])

{'bleu': 0.0,
 'precisions': [0.0, 0.0, 0.0, 0.0],
 'brevity_penalty': 0.01831563888873418,
 'length_ratio': 0.2,
 'translation_length': 1,
 'reference_length': 5}

In [44]:
res = rouge.compute(predictions=predictions, references=references)
res

{'rouge1': 0.22743817069702782,
 'rouge2': 0.14307026382112265,
 'rougeL': 0.22275573567627416,
 'rougeLsum': 0.22435905485779456}

In [45]:
bleus = []

for pred, ref in zip(predictions.values, references.values):
    b = rouge.compute(predictions=[pred], references=[ref])
    bleus.append(b)

In [46]:
res = pd.DataFrame(bleus)
res

,rouge1,rouge2,rougeL,rougeLsum
0,0.333333,0.200000,0.333333,0.333333
1,0.666667,0.600000,0.666667,0.666667
2,1.000000,1.000000,1.000000,1.000000
3,0.047619,0.000000,0.047619,0.047619
4,0.122449,0.085106,0.122449,0.122449
...,...,...,...,...
282,0.111111,0.000000,0.055556,0.055556
283,0.081633,0.000000,0.081633,0.081633
284,0.000000,0.000000,0.000000,0.000000
285,1.000000,0.000000,1.000000,1.000000


In [47]:
import matplotlib as plt
res.describe()

,rouge1,rouge2,rougeL,rougeLsum
count,287.000000,287.000000,287.000000,287.000000
mean,0.227927,0.143195,0.223621,0.223621
std,0.324181,0.296095,0.323932,0.323932
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.076923,0.000000,0.074074,0.074074
75%,0.323265,0.108187,0.323265,0.323265
max,1.000000,1.000000,1.000000,1.000000


In [51]:
res['max'] = res.max(axis=1)

In [61]:
df.loc[res['max'] < 0.2, ['questions','answers','generated_answers','contexts','retrieved_docs']].to_excel('answer_file_9_error_analysis.xlsx',index=False)

In [74]:
df.loc[res['max'] < 0.2].loc[13].questions, df.loc[res['max'] < 0.2].loc[13].answers

('What is the most effective treatment against influenza?', 'Vaccination')

In [75]:
df.loc[res['max'] < 0.2].loc[13].generated_answers

'neuraminidase inhibitor'

In [106]:
import re

def preprocess_text(text: str)->str:
    
    # remove any big crump of strings
    text = re.sub(r"\S{20,}","",text)

    # remove citation numbers
    text = re.sub(r"\[(\d,?\s?-?)+\]","",text)
    
    # remove percentage in parenthses e.g. (20%)
    text = re.sub(r"\((\d+%;?\s?)+\)","",text)

    # remove everything in any parenthesis
    text = re.sub(r"\(([^)]+)\)","",text)

    # remove all Latex tags
    text = re.sub(r"\\(\w+)([\{\[][^}]+\})+","",text)

    # remove big non-character crumps (including special characters like \n, \t)
    text = re.sub(r"[^a-zA-Z]{20,}","",text)

    return text

with open('../../data/covid/PMC3979752.xml.json.txt','r') as f:
    doc = f.read()

In [108]:
doc

"Vaccination remains most economical and effective means against respiratory diseases caused by influenza viruses [1]. Based on the circulating viruses in the population, trivalent vaccine strains have been developed and are used for the influenza virus protection [2]. The most acceptable current available strategy is the intramuscular administration of inactivated vaccines produced by egg-based manufacturing systems which while effective, are hampered by limited capacity and flexibility [3]. However, vaccine strains must be frequently adapted to match the circulating viruses throughout the world [4]. In addition, the levels of antibody induced by the inactivated vaccine have been observed to decrease by 75% over an 8-month period [2], [5]. Therefore, alternative strategies for developing broadly cross-protective, safe and effective vaccines against influenza viral infections are of prominent importance. Matrix protein 2 (M2) is highly conserved among influenza A virus strains, indicat

In [107]:
preped_doc = preprocess_text(doc)
preped_doc

"Vaccination remains most economical and effective means against respiratory diseases caused by influenza viruses . Based on the circulating viruses in the population, trivalent vaccine strains have been developed and are used for the influenza virus protection . The most acceptable current available strategy is the intramuscular administration of inactivated vaccines produced by egg-based manufacturing systems which while effective, are hampered by limited capacity and flexibility . However, vaccine strains must be frequently adapted to match the circulating viruses throughout the world . In addition, the levels of antibody induced by the inactivated vaccine have been observed to decrease by 75% over an 8-month period , . Therefore, alternative strategies for developing broadly cross-protective, safe and effective vaccines against influenza viral infections are of prominent importance. Matrix protein 2  is highly conserved among influenza A virus strains, indicating that M2 is an attr

In [92]:
import textwrap

print(textwrap.fill(preped_doc, 100))

Vaccination remains most economical and effective means against respiratory diseases caused by
influenza viruses . Based on the circulating viruses in the population, trivalent vaccine strains
have been developed and are used for the influenza virus protection . The most acceptable current
available strategy is the intramuscular administration of inactivated vaccines produced by egg-based
manufacturing systems which while effective, are hampered by limited capacity and flexibility .
However, vaccine strains must be frequently adapted to match the circulating viruses throughout the
world . In addition, the levels of antibody induced by the inactivated vaccine have been observed to
decrease by 75% over an 8-month period , . Therefore, alternative strategies for developing broadly
cross-protective, safe and effective vaccines against influenza viral infections are of prominent
importance. Matrix protein 2  is highly conserved among influenza A virus strains, indicating that
M2 is an attra

In [89]:
import textwrap


text = df.loc[res['max'] < 0.2].loc[13].contexts
print(textwrap.fill(text, 100))

Mucosal Vaccination with Recombinant Lactobacillus casei-Displayed CTA1-Conjugated Consensus Matrix
Protein-2 (sM2) Induces Broad Protection against Divergent Influenza Subtypes in BALB/c Mice
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3979752/  SHA: efaa556b484fbcd9cc34832ffac53ef3e834e9c0
Authors: Chowdhury, Mohammed Y. E.; Li, Rui; Kim, Jae-Hoon; Park, Min-Eun; Kim, Tae-Hwan;
Pathinayake, Prabuddha; Weeratunga, Prasanna; Song, Man Ki; Son, Hwa-Young; Hong, Seung-Pyo; Sung,
Moon-Hee; Lee, Jong-Soo; Kim, Chul-Joong Date: 2014-04-08 DOI: 10.1371/journal.pone.0094051 License:
cc-by  Abstract: To develop a safe and effective mucosal vaccine against pathogenic influenza
viruses, we constructed recombinant Lactobacillus casei strains that express conserved matrix
protein 2 with (pgsA-CTA1-sM2/L. casei) or without (pgsA-sM2/L. casei) cholera toxin subunit A1
(CTA1) on the surface. The surface localization of the fusion protein was verified by cellular
fractionation analyses, flow cytometr

In [9]:
import pandas as pd

df_new_prompt = pd.read_excel('answer_file_13.xlsx')

In [13]:
(df_new_prompt.sts > 0.5).sum() / len(df_new_prompt)

0.41114982578397213